# Homework 7 Link analysis

The url I choose is https://doc.scrapy.org/en/latest/intro/tutorial.html. 
I wrote a crawler to grab first 200 URLs. And this crawler considers the complications in '<a href'. More details will be commented in the code.

In [1]:
import numpy as np
import requests
from scrapy.http import TextResponse
from numpy import *

7.1 Create a crawler using the webcrawler provided and discussed in my jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) , have it crawl the first 200 pages from a base URL of your choosin

In [2]:
#crawl from url and update matrix if needed
#fidx is the index of url in urllist

def crawl(fidx,url,urllist,matrix):
    idx=len(urllist)
    i=url.rfind('/')
    me=url[0:i+1]  #current level 
    i=url.rfind('/',0,i-1)
    parent=url[0:i+1] # get the upper level
    i=url.find('://')
    r=requests.get(url)
    response= TextResponse(r.url, body=r.text, encoding='utf-8')
    link=response.xpath('//ul/li/a/@href').extract()  # all url link
    for x in link:
        if x.endswith('://'): # get rid of '://' illeagal format
            continue
        elif x.startswith('../'):
            x=parent+x[3:]
            idx=addtolist(fidx,idx,x,urllist, matrix)
            if idx==200:
                return idx
        elif x.startswith('#'): # point to url itself, but same page
            matrix[fidx][fidx]=1
        elif x.startswith('http'):
            idx=addtolist(fidx,idx,x,urllist,matrix)
            if idx==200:
                return idx
        else:
            x=me+x
            idx=addtolist(fidx,idx,x,urllist,matrix)
            if idx==200:
                return idx
    return idx

In [3]:
#add url x to the list, updata matrix while crawling 
#fidx is the index of source url in urllist
#idx is the index of of next free page url in urllist
def addtolist(fidx,idx,x,url,matrix):
    i=x.rfind('#')  # remove the last '#something...', because it's just pointing to some section in the html page
    if i!=-1:
        x=x[:i]  # get rid of '#'
    if x not in urllist:  # a new page
        urllist.append(x)
        matrix[fidx][idx]=1
        idx+=1
    else:   # a old page
        k=urllist.index(x) # find the index of old page
        matrix[fidx][k]=1
    return idx

In [4]:
urllist=['https://www.olx.com.pk/']  #the starting url
matrix=np.zeros((200,200)) #the matrix
num=0
fidx=0
while num<200 and fidx<len(urllist):
    num=crawl(fidx,urllist[fidx],urllist,matrix)
    fidx+=1
print 'The first 200 URLs are:'
for x in urllist:
    print x    


The first 200 URLs are:
https://www.olx.com.pk/
https://www.olx.com.pk//tablets_c1455
https://www.olx.com.pk//accessories_c1457
https://www.olx.com.pk//mobile-phones_c1453
https://www.olx.com.pk//cars_c84
https://www.olx.com.pk//cars-accessories_c2027
https://www.olx.com.pk//spare-parts_c82
https://www.olx.com.pk//buses-vans-trucks_c85
https://www.olx.com.pk//land-plots_c40
https://www.olx.com.pk//houses_c1721
https://www.olx.com.pk//apartments-flats_c1725
https://www.olx.com.pk//shops-offices-commercial-space_c1733
https://www.olx.com.pk//houses_c1719
https://www.olx.com.pk//apartments-flats_c1723
https://www.olx.com.pk//portions-floors_c39
https://www.olx.com.pk//shops-offices-commercial-space_c1731
https://www.olx.com.pk//computers-accessories_c443
https://www.olx.com.pk//tv-video-audio_c729
https://www.olx.com.pk//cameras-accessories_c727
https://www.olx.com.pk//games-entertainment_c93
https://www.olx.com.pk//motorcycles_c81
https://www.olx.com.pk//spare-parts_c1587
https://www.olx

7.2  Create a Stochastic matrix from its resulting crawling

In [11]:
def pagerank(H):
    n= len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
        if multiply.reduce(H[i]== zeros(n)):
            w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    m = 1
    j = 0
    while m > 0.000001:  #until it converges
        rho1 = dot(rho,G)
        m = np.linalg.norm(rho1-rho)  
        rho = rho1
        j+=1
    print 'pagerank converge after %d loops'%(j)
    return rho

In [12]:
page_matrix=matrix.copy()
for i in range(len(matrix)):
    k=sum(matrix[i])
    if k>0:
        page_matrix[i]/=k
        
H=pagerank(page_matrix)
print H

pagerank converge after 3 loops
[0.00487426 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218 0.00494218
 0.00494218 0.00494218 0.00532278 0.00532278 0.00532278 0.00532278
 0.00532278 0.00494218 0.00541045 0.00516735 0.00516735 0.00516735
 0.00525486 0.00497196 0.00497196 0.00497196 0.00497196 0.00497196
 0.00497196 0.00497196 0.00497196 0.00497196 0.00497196 0.00497196
 0.00497196 0.00497196 0.00497196 0.00497196 0.00497196 0.00497196
 0.00497196 0.00497196 0.00497

7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in your sample

In [7]:
v=np.argsort(-H)
print 'The top 5 page URL:'
for i in range(5):
    print urllist[v[i]]

The top 5 page URL:
https://www.olx.com.pk///items
https://www.olx.com.pk///cars_c84
https://www.olxgroup.com/
https://help.olx.com.pk/hc/en-us/requests/new?ticket_form_id=29191
https://bit.ly/olxsales


7.4 For extra credit use the hits algorithm ( with a connectivity matrix 

In [13]:
def hits(A):
    n= len(A)
    Au= dot(transpose(A),A)
    Hu = dot(A,transpose(A))
    a = ones(n); h = ones(n)    
    m = 1
    j = 0
    while m > 0.000001:  #until it converges
        a1 = dot(a,Au)
        a1= a1/sum(a1)
        h1 = dot(h,Hu)
        h1 = h1/ sum(h1)
        m = np.linalg.norm(a1-a)+ np.linalg.norm(h1-h)   
        a=a1
        h=h1
        j+=1
    print 'Hits converge after %d loops'%(j)
    return a
        

In [14]:
H1=hits(matrix)
print H1

Hits converge after 22 loops
[0.         0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479 0.00253479
 0.00253479 0.00253479 0.02099144 0.02099144 0.02099144 0.02099144
 0.02099144 0.00253479 0.01951028 0.0137979  0.0137979  0.0137979
 0.01845665 0.0045993  0.0045993  0.0045993  0.0045993  0.0045993
 0.0045993  0.0045993  0.0045993  0.0045993  0.0045993  0.0045993
 0.0045993  0.0045993  0.0045993  0.0045993  0.0045993  0.0045993
 0.0045993  0.0045993  0.0045993  0.0

I found that if the data is not that big, the different between pagerank and hits algorithm is small. 

In [15]:
v1=np.argsort(-H1)
print 'The 5 top page URL:'
for i in range(5):
    print urllist[v1[i]]

The 5 top page URL:
http://help.olx.com.pk/
https://bit.ly/olxsales
https://help.olx.com.pk/hc/en-us/requests/new?ticket_form_id=29191
https://www.olxgroup.com/careers
https://www.olxgroup.com/
